In [74]:

# import matplotlib
from matplotlib import pyplot as plt
import numpy as np 
import pandas as pd 

# import sklearn
# from sklearn import linear_model
from sklearn.linear_model import Lasso, Ridge, LogisticRegression, LogisticRegressionCV
from sklearn.preprocessing import StandardScaler,LabelEncoder,OneHotEncoder, PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,r2_score, confusion_matrix
import numpy.random as r

Preprocessing the data!

In [75]:
file_path = "rf_signal_data.csv"
rf_data = pd.read_csv(file_path)

#check how the data looks
#print(rf_data)
dnnd = 7
#Let's hot encode the modulation types, weather conditions, interference type, Antenna Type, and device status using integers
label_encoder = LabelEncoder()
rf_data['Modulation_Original'] = rf_data['Modulation']
rf_data['Modulation'] = label_encoder.fit_transform(rf_data['Modulation']) + 1  # Add 1 to avoid 0 indexing
rf_data['Interference Type'] = label_encoder.fit_transform(rf_data['Interference Type']) + 1
rf_data['Weather Condition'] = label_encoder.fit_transform(rf_data['Weather Condition']) + 1
rf_data['Device Status'] = label_encoder.fit_transform(rf_data['Device Status']) + 1
rf_data['Antenna Type'] = label_encoder.fit_transform(rf_data['Antenna Type']) + 1
rf_data['Device Type'] = label_encoder.fit_transform(rf_data['Device Type']) + 1
# print(rf_data)

X = rf_data.loc[:,~rf_data.columns.isin(['Device Status','Modulation','Timestamp','Location','Latitude','Longitude','Altitude','Air Pressure','I/Q Data','Modulation_Original','Weather Condition','Antenna Type', 'Battery Level' ])]
y = rf_data['Modulation']

#Split the data into test data and validation data
X_train, X_test, y_train, y_test = train_test_split( X, y,random_state=42)

scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# print(X)
# print(X_train)

# Supervised Analysis I: Logistic Regression

In [76]:
# trainingError, validationError, variance, recall = [[1, 2, 3, 4, 5],[1, 2, 3, 4, 5],[1, 2, 3, 4, 5]], [], [], []
# for each in overfitting(3):
#     for each in regularization(5):
#         train the Model
#         trainingError[overfit][regular] = training error of this model



Process the data so that we can perform logistic regression:

In [77]:
ones = np.ones((X_train.shape[0],1))
ones2 = np.ones((X_test.shape[0],1))
X_train = np.hstack((ones,X_train))
X_test = np.hstack((ones2,X_test))
y_train = np.array(y_train).reshape(-1, 1)
y_test = np.array(y_test).reshape(-1, 1)

In [80]:
# import numpy as np
# import matplotlib.pyplot as plt
# import seaborn as sns

# class_names=['1', '2', '3', '4', '5', '6'] # name  of classes
# fig, ax = plt.subplots()
# tick_marks = np.arange(len(class_names))
# plt.xticks(tick_marks, class_names)
# plt.yticks(tick_marks, class_names)
# # create heatmap
# sns.heatmap(pd.DataFrame(cnf_matrix), annot=True, cmap="YlGnBu" ,fmt='g')
# ax.xaxis.set_label_position("top")
# plt.tight_layout()
# plt.title('Confusion matrix', y=1.1)
# plt.ylabel('Actual label')
# plt.xlabel('Predicted label')

# plt.Text(0.5,257.44,'Predicted label')

In [104]:
XTrainPolynomialDegrees = []
XTestPolynomialDegrees = []
nMin = 3
nMax = 3
for i in range (nMin, nMax+1):
    poly = PolynomialFeatures(i)
    XTrainPolynomialDegrees.append(poly.fit_transform(X_train))
    XTestPolynomialDegrees.append(poly.fit_transform(X_test))

In [ ]:
# iters = np.array(range(0,num_iters,10))
# # iters = np.array(range(num_iters))
# plt.plot(iters,log_likelihood_values,'.-',color='green')
# plt.xlabel('Number of iterations')
# plt.ylabel('Log-Likelihood')
# plt.title("Log-Likelihood vs Number of Iterations.")
# plt.grid()
# plt.show(block=False)

In [105]:
CsMin = 1
CsMax = 4
dict = {}
for j in range(CsMin, CsMax+1):
    dict[j]={}
    for i in range(len(XTrainPolynomialDegrees)):
        logreg = LogisticRegressionCV(Cs =j,random_state=32, multi_class='multinomial', penalty = 'l2',max_iter=100)
        logreg.fit(XTrainPolynomialDegrees[i], y_train)
        y_pred = logreg.predict(XTestPolynomialDegrees[i])
        training_error = 1-logreg.score(XTrainPolynomialDegrees[i],y_train)
        validation_error = 1-logreg.score(XTestPolynomialDegrees[i],y_test)
        cnf_matrix = confusion_matrix(y_test, y_pred)
        dict[j][i] = [training_error, validation_error, cnf_matrix]

        
        

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/utils/validation.py:1229: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.19964262508122158
0.16937134502923976


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/utils/validation.py:1229: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


KeyboardInterrupt: 

In [ ]:
# cnf_matrix = confusion_matrix(y_test, y_pred)
